This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5) and depends on Magma (tested using version
2.26-9).

In this notebook, we complete the identification of generic curves of genus 7 with 7 $\mathbb{F}_2$-points which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 20 minutes for completion.

In [1]:
load("preamble.sage")

Construct the quadratic relations cutting out the orthogonal Grassmannian $OG$.

In [2]:
F = GF(2)
P.<x0,x12,x13,x14,x15,x23,x24,x25,x34,x35,x45,x1234,x1235,x1245,x1345,x2345> = PolynomialRing(F, 16)
quads = [x0*x2345 + x23*x45 + x24*x35 + x25*x34,
         x12*x1345 + x13*x1245 + x14*x1235 + x15*x1234,
         x0*x1345 + x13*x45 + x14*x35 + x15*x34,
         x12*x2345 + x23*x1245 + x24*x1235 + x25*x1234,
         x0*x1245 + x12*x45 + x14*x25 + x15*x24,
         x13*x2345 + x23*x1345 + x34*x1235 + x35*x1234,
         x0*x1235 + x12*x35 + x13*x25 + x15*x23,
         x14*x2345 + x24*x1345 + x34*x1245 + x45*x1234,
         x0*x1234 + x12*x34 + x13*x24 + x14*x23,
         x15*x2345 + x25*x1345 + x35*x1245 + x45*x1235]

In [3]:
def linear_section(coords):
    V = Matrix(coords).right_kernel()
    return [sum(P.gens()[i] * v[-1-i] for i in range(16)) for v in V.gens()] + quads

Read in data about 6-tuples of points on $OG$.

In [4]:
with open("6-tuples.txt", "r") as f:
    s = f.read()
    l = sage_eval(s)
    l = [[vector(F, v) for v in vecs] for vecs in l]
len(l)

494

Confirm that in all 6-tuples whose linear span is not 6-dimensional, there is no 7th point of $OG$ in the linear span (which means we can start with a different 6-tuple whose linear span is 6-dimensional).

In [5]:
l2 = [vecs for vecs in l if Matrix(F, vecs).rank() < 6]
len(l2)

3

In [6]:
for vecs in l2:
    gens = linear_section(vecs)
    J = P.ideal(gens)
    tmp = J.primary_decomposition()
    assert len([(J2.radical().hilbert_polynomial(), J2.hilbert_polynomial()) for J2 in tmp]) <= 6

Remove cases where the linear span is not 6-dimensional. Without loss of generality, we can always start with one of the remaining cases.

In [7]:
l2 = [vecs for vecs in l if Matrix(F, vecs).rank() == 6]
len(l2)

491

In [8]:
proj = ProjectiveSpace(P)
OG = proj.subscheme(quads)
rp = [vector(pt) for pt in OG.rational_points()]
len(rp)

2295

Identify extensions of a given 6-tuple to a 7-tuple whose linear span contains no other $\mathbb{F}_2$-points of $OG$.

In [9]:
l3a = []
for vecs in l:
    d = defaultdict(list)
    for pt in rp:
        if pt not in vecs:
            M = Matrix(F, vecs + [pt]).echelon_form()
            M.set_immutable()
            d[M].append(pt)
    for M in d:
        if len(d[M]) == 1:
            vecs2 = vecs + d[M]
            if Matrix(F, vecs2).rank() == 7:
                l3a.append(linear_section(vecs2))
len(l3a)

123606

Identify 6-tuples for which the linear span contains a 7th $\mathbb{F}_2$-rational point of $OG$ *and* either:
- the linear span is transversal to $OG$ at all 7 of the $\mathbb{F}_2$-rational points; or
- the highest order of tangency among $\mathbb{F}_2$-rational points occurs for the point not listed in the original 6-tuple.


In [10]:
l2a = []
l2b = []
for vecs in l:
    gens = linear_section(vecs)
    J = P.ideal(gens)
    if J.dimension() > 1:
        continue
    tmp = J.primary_decomposition()
    tmp2 = [(J2.radical().hilbert_polynomial(), J2.hilbert_polynomial()) for J2 in tmp]
    if sum(1 for (a,b) in tmp2 if a==1) >= 7:
        m = max(b for (a,b) in tmp2 if a==1)
        if m == 1:
            l2a.append(vecs)
        else:
            tmp3 = [i for i in range(len(tmp)) if tmp2[i][0] == 1]
            tmp3 = [i for i in tmp3 if all(tmp[i].radical() != P.ideal(linear_section([v])) for v in vecs)]
            assert len(tmp3) == 1
            if tmp2[tmp3[0]][1] == m:
                l2b.append(vecs)
len(l2a), len(l2b)

(2, 42)

Compute 7-dimensional extensions of the 6-dimensional linear spans, then enforce the condition that $\#C(\mathbb{F}_2) = 7$.

In [11]:
l3b = []
V0 = VectorSpace(F, 16)
for vecs in l2a + l2b:
    for (v,) in subspaces_containing(V0, V0.subspace(vecs), 1, basis=True):
        l3b.append(linear_section(vecs + [v]))
len(l3b)

45012

In [12]:
def count_by_ideal(gens, n):
    J = P.ideal(gens + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [13]:
l3c = [gens for gens in l3b if count_by_ideal(gens, 1) == 7]
len(l3c)

7092

Merge the two lists, then enforce the conditions that $\#C(\mathbb{F}_4) = 15$ and $\#C(\mathbb{F}_8) = 7$.

In [14]:
l4 = [gens for gens in l3a+l3c if count_by_ideal(gens, 2) == 15]
len(l4)

5038

In [15]:
l5 = [gens for gens in l4 if count_by_ideal(gens, 3) == 7]
len(l5)

447

Close out this case.

In [16]:
proj = magma.ProjectiveSpace(P)
curves = {(7,15,7): l5}
closeout(curves, X=proj, genus=7)

Number of curves found: 0
Number of isomorphism classes found: 0
No covers found in this case!
Total time: 19 minutes
